# Experiment: Which drugs are "similar", from prescription data?

In the USA the social program for prescription drugs is the "Medicare Part D Prescription Drug" program. Data regarding the prescribed drugs per provider and drug are available at https://data.cms.gov/provider-summary-by-type-of-service/medicare-part-d-prescribers/medicare-part-d-prescribers-by-provider-and-drug/

On this website you can select the columns you want. I downloaded only the code for the drug provider (`Prscrbr_NPI`), the type of provider (`Prscrbr_Type`), the generic name for the drug (`Gnrc_Name`) and the total number of claims made by this provider (`Tot_Clms`):

<center><img src='./image.png' width=50%></img></center>

As a result, I got a 197MB zip file. I made a folder named `data` and placed the file there. Let's load the data:

In [6]:
import zipfile
from pathlib import Path

import pandas as pd

DATA_DIR = Path.cwd() / 'data'
CSV_FILE_NAME = 'Medicare_Part_D_Prescribers_by_Provider_and_Drug_2022.csv'



def load_data(data_dir=DATA_DIR, csv_file_name=CSV_FILE_NAME):
    csv_file_path = data_dir / csv_file_name

    if not csv_file_path.is_file():
        print(f'{csv_file_path} not found, extracting zip file')
        zip_file_path = data_dir / "Medicare_Part_D_Prescribers_by_Provider_and_Drug_2022.zip"
        if not zip_file_path.is_file():
            raise FileNotFoundError(f"File not found: {zip_file_path}")
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(data_dir)
    print(f'Loading data from {csv_file_path}')
    df = pd.read_csv(csv_file_path)
    return df

In [7]:
df = load_data()

Loading data from c:\Users\brnos\OneDrive\Documentos\insper\6comp\MachineLearning\machine_learning\labs\01_SVD\data\Medicare_Part_D_Prescribers_by_Provider_and_Drug_2022.csv


## Exploratory analysis

What does the data look like?

In [8]:
df.shape

(25869521, 4)

- About 25 million rows, 4 columns

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25869521 entries, 0 to 25869520
Data columns (total 4 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   Prscrbr_NPI   int64 
 1   Prscrbr_Type  object
 2   Gnrc_Name     object
 3   Tot_Clms      int64 
dtypes: int64(2), object(2)
memory usage: 789.5+ MB


- The columns are:

| Column Name  | Description                  | Dtype   | Type meaning |
|--------------|------------------------------|---------|--------------|
| Prscrbr_NPI  | Doctor's ID                  | int64   | an integer   |  
| Prscrbr_Type | Doctor type                  | object  | a string     | 
| Gnrc_Name    | Drug name                    | object  | a string     | 
| Tot_Clms     | Number of prescriptions made | int64   | an integer   | 

In [10]:
df.head(10)

,Prscrbr_NPI,Prscrbr_Type,Gnrc_Name,Tot_Clms
0,1003000126,Internal Medicine,Amlodipine Besylate,19
1,1003000126,Internal Medicine,Atorvastatin Calcium,11
2,1003000126,Internal Medicine,Apixaban,15
3,1003000126,Internal Medicine,Escitalopram Oxalate,16
4,1003000126,Internal Medicine,Hydrochlorothiazide,12
5,1003000126,Internal Medicine,Levofloxacin,11
6,1003000126,Internal Medicine,Levothyroxine Sodium,11
7,1003000126,Internal Medicine,Losartan Potassium,13
8,1003000126,Internal Medicine,Metoprolol Succinate,12
9,1003000126,Internal Medicine,Pantoprazole Sodium,24


How many absent items (NaNs) are there in the dataset?

In [11]:
df.isna().sum()

Prscrbr_NPI     0
Prscrbr_Type    5
Gnrc_Name       0
Tot_Clms        0
dtype: int64

So there are only five absent items in the `Prscrbr_Type` column. Let's discard the corresponding rows:

In [12]:
df = df.dropna()

In [13]:
df.shape

(25869516, 4)

The `Prscrbr_NPI`, `Prscrbr_Type`, and `Gnrc_Name` contain categorical values. How many categories in each of them?

In [14]:
df[['Prscrbr_NPI', 'Prscrbr_Type', 'Gnrc_Name']].nunique()

Prscrbr_NPI     1057564
Prscrbr_Type        180
Gnrc_Name          1757
dtype: int64

If we were to create a matrix of `Prscrbr_NPI` by `Gnrc_Name`, with `Tot_Clms` for cell values, that matrix would be of size $1057564 \times 1757$! How many values a matrix of this size would store?

In [19]:
num_positions = 1057564 * 1757
print(f'{num_positions:,}')

1,858,139,948


Almost two billion positions! And most of them are zeros, because we only have $25,869,516$ non-zero values.

In [20]:
num_nonzero_positions = df.shape[0]
print(f'{num_nonzero_positions:,}')

25,869,516


The occupancy ratio of this matrix is very, very small:

In [21]:
occupancy_ratio = num_nonzero_positions / num_positions
print(f'{occupancy_ratio:.2%}')

1.39%


Only about $1.39\%$ of the matrix is non-zero. Such matrices are called *sparse matrices*. There are algorithmic techniques to handle sparse matrices in an efficient manner, both in terms of amount of computation to be performed and in memory usage. We will make use of it in a bit.

But we should also consider whether all of these categories in `Prscrbr_NPI` and `Gnrc_Name` are truly useful. Maybe categories with under-representation could be discarded, since we want to focus on the most present information only.

Let's start with the physician type:

In [22]:
count_Prscrbr_Type = df['Prscrbr_Type'].value_counts()

In [23]:
count_Prscrbr_Type.head(10)

Prscrbr_Type
Family Practice        6651765
Internal Medicine      5603062
Nurse Practitioner     5053595
Physician Assistant    1878231
Cardiology              710812
Psychiatry              547364
General Practice        407701
Neurology               386005
Endocrinology           271592
Ophthalmology           260765
Name: count, dtype: int64

There is a steep decline in representation from the most common type of doctor ("Family Practice") to the 10th most common ("Ophtalmology").

In [24]:
count_Prscrbr_Type.tail(10)

Prscrbr_Type
Clinical Medical Laboratory                              1
Assisted Living Facility                                 1
Public Health or Welfare Agency                          1
Home Health Aide                                         1
Local Education Agency (LEA)                             1
Independent Diagnostic Testing Facility (IDTF)           1
Nursing Home Administrator                               1
Doula                                                    1
Residential Treatment Facility, Physical Disabilities    1
Phlebology                                               1
Name: count, dtype: int64

The least represented types of doctor have only one entry each.

Let's decide on a cutoff value to filter the least common types. It will be quite arbitrary, and it is worthwhile to consider whether this removal of information will qualitatively impact the conclusions to be obtained from the analysis of this dataset - keep this in mind, ok?

In [25]:
import numpy as np

In [26]:
count_Prscrbr_Type.quantile(np.linspace(0, 1, 21))

0.00          1.00
0.05          1.00
0.10          2.00
0.15          6.00
0.20         11.00
0.25         19.00
0.30         28.00
0.35         46.30
0.40        124.20
0.45        193.30
0.50        465.50
0.55       1169.85
0.60       2337.00
0.65       4591.80
0.70       8970.40
0.75      18265.25
0.80      50342.20
0.85      99978.15
0.90     156544.30
0.95     261306.35
1.00    6651765.00
Name: count, dtype: float64

Making a list of quantiles from zero to $100\%$, in steps of $5\%$, we see an almost exponential growth in number of physicians per type. Let's cut aggressively then: it will not change the number of data points substantially.

In [27]:
q_Prscrbr_Type = 0.70
cut_value = count_Prscrbr_Type.quantile(q_Prscrbr_Type)

useful_Prscrbr_Type = count_Prscrbr_Type[count_Prscrbr_Type >= cut_value].index

In [28]:
useful_Prscrbr_Type

Index(['Family Practice', 'Internal Medicine', 'Nurse Practitioner',
       'Physician Assistant', 'Cardiology', 'Psychiatry', 'General Practice',
       'Neurology', 'Endocrinology', 'Ophthalmology', 'Nephrology',
       'Emergency Medicine', 'Dentist', 'Pulmonary Disease',
       'Gastroenterology', 'Hematology-Oncology', 'Dermatology', 'Urology',
       'Rheumatology', 'Hospitalist', 'Interventional Cardiology',
       'Geriatric Medicine',
       'Student in an Organized Health Care Education/Training Program',
       'Optometry', 'Obstetrics & Gynecology', 'Psychiatry & Neurology',
       'Orthopedic Surgery', 'Infectious Disease',
       'Physical Medicine and Rehabilitation', 'Otolaryngology', 'Podiatry',
       'General Surgery', 'Medical Oncology', 'Allergy/ Immunology',
       'Clinical Cardiac Electrophysiology', 'Pharmacist', 'Anesthesiology',
       'Pain Management', 'Pediatric Medicine',
       'Certified Clinical Nurse Specialist', 'Interventional Pain Management',
    

In [29]:
df = df[df['Prscrbr_Type'].isin(useful_Prscrbr_Type)]

In [30]:
df.shape

(25754507, 4)

We only reduced the number of data points from $25,869,516$ to $25,754,507$!

Now let's look into the physicians:

In [31]:
count_Prscrbr_NPI = df['Prscrbr_NPI'].value_counts()

In [32]:
count_Prscrbr_NPI.quantile(np.linspace(0, 1, 21))

0.00      1.0
0.05      1.0
0.10      1.0
0.15      1.0
0.20      2.0
0.25      2.0
0.30      3.0
0.35      4.0
0.40      5.0
0.45      6.0
0.50      8.0
0.55     10.0
0.60     13.0
0.65     17.0
0.70     22.0
0.75     29.0
0.80     39.0
0.85     53.0
0.90     75.0
0.95    111.0
1.00    590.0
Name: count, dtype: float64

Again, a very exponential-like growth of number of prescriptions per physician. Let's cut aggressively again, but remember: this may impact the quality of the subsequent analysis, so it may be interesting to revisit these arbitrary decisions later.

In [33]:
q_Prscrbr_NPI = 0.5
cut_value = count_Prscrbr_NPI.quantile(q_Prscrbr_NPI)

useful_Prscrbr_NPI = count_Prscrbr_NPI[count_Prscrbr_NPI >= cut_value].index

In [34]:
len(useful_Prscrbr_NPI)

537484

In [35]:
df = df[df['Prscrbr_NPI'].isin(useful_Prscrbr_NPI)]

In [36]:
df.shape

(24297207, 4)

We are keeping a large portion of the information still: $24,297,207$, from the previous number of $25,754,507$ rows.

Now let's investigate the drugs:

In [37]:
count_Gnrc_Name = df['Gnrc_Name'].value_counts()

In [38]:
count_Gnrc_Name.head(10)

Gnrc_Name
Levothyroxine Sodium    399226
Metformin Hcl           383512
Gabapentin              315303
Atorvastatin Calcium    307186
Albuterol Sulfate       301663
Amlodipine Besylate     299293
Lisinopril              286633
Losartan Potassium      270682
Omeprazole              260114
Furosemide              256761
Name: count, dtype: int64

In [39]:
count_Gnrc_Name.tail(10)

Gnrc_Name
Pediatric Multivitamin No.17      1
Opium/Belladonna Alkaloids        1
Ethacrynate Sodium                1
Mannitol                          1
Carglumic Acid                    1
Lansoprazole/Amoxiciln/Clarith    1
Acetaminophen                     1
Methotrexate                      1
Secnidazole                       1
Trifarotene                       1
Name: count, dtype: int64

In [40]:
count_Gnrc_Name.quantile(np.linspace(0, 1, 21))

0.00         1.00
0.05         1.00
0.10         3.00
0.15         7.00
0.20        13.00
0.25        23.00
0.30        37.00
0.35        59.00
0.40       111.00
0.45       187.25
0.50       303.00
0.55       601.75
0.60       998.00
0.65      1627.50
0.70      2728.00
0.75      4894.50
0.80      7601.00
0.85     14959.00
0.90     33058.50
0.95     93739.25
1.00    399226.00
Name: count, dtype: float64

Again, an exponential growth in quantiles, let's cut:

In [41]:
q_Gnrc_Name = 0.5
cut_value = count_Gnrc_Name.quantile(q_Gnrc_Name)

useful_Gnrc_Name = count_Gnrc_Name[count_Gnrc_Name >= cut_value].index

In [42]:
useful_Gnrc_Name

Index(['Levothyroxine Sodium', 'Metformin Hcl', 'Gabapentin',
       'Atorvastatin Calcium', 'Albuterol Sulfate', 'Amlodipine Besylate',
       'Lisinopril', 'Losartan Potassium', 'Omeprazole', 'Furosemide',
       ...
       'Alectinib Hcl', 'Sulfacetamide Sodium', 'Fondaparinux Sodium',
       'Sub-Q Insulin Device, 30 Unit', 'Trametinib Dimethyl Sulfoxide',
       'Chlordiazepoxide/Clidinium Br', 'Erythromycin Base In Ethanol',
       'Lamivudine/Zidovudine', 'Emtricita/Rilpivirine/Tenof Df',
       'Dipyridamole'],
      dtype='object', name='Gnrc_Name', length=875)

In [43]:
df = df[df['Gnrc_Name'].isin(useful_Gnrc_Name)]

In [44]:
df.shape

(24246835, 4)

Finally, after all this filtering, we went from $25,869,521$ items to $24,246,835$.

In [45]:
print(f'We dropped {1.0 - 24246835/25869521:.2%} of the data')

We dropped 6.27% of the data


In [46]:
df['Prscrbr_NPI'].nunique()

537484

In [47]:
df['Gnrc_Name'].nunique()

875

In [48]:
318660 * 857

273091620

In [49]:
df.shape

(24246835, 4)

In [50]:
21400306 / 273091620

0.07836310026649665

create a sparse matrix from the dataframe

In [51]:
df = df.reset_index(drop=True)

In [52]:
df['Prscrbr_NPI'] = df['Prscrbr_NPI'].astype('category')
df['Gnrc_Name'] = df['Gnrc_Name'].astype('category')
df['Prscrbr_Type'] = df['Prscrbr_Type'].astype('category')
df['Tot_Clms'] = df['Tot_Clms'].astype(float)

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24246835 entries, 0 to 24246834
Data columns (total 4 columns):
 #   Column        Dtype   
---  ------        -----   
 0   Prscrbr_NPI   category
 1   Prscrbr_Type  category
 2   Gnrc_Name     category
 3   Tot_Clms      float64 
dtypes: category(3), float64(1)
memory usage: 367.1 MB


In [54]:
df['Gnrc_Name'].cat.categories

Index(['0.9 % Sodium Chloride', 'Abacavir Sulfate',
       'Abacavir Sulfate/Lamivudine', 'Abacavir/Dolutegravir/Lamivudi',
       'Abaloparatide', 'Abatacept', 'Abemaciclib', 'Abiraterone Acetate',
       'Acalabrutinib', 'Acamprosate Calcium',
       ...
       'Voriconazole', 'Vortioxetine Hydrobromide', 'Warfarin Sodium',
       'Zafirlukast', 'Zaleplon', 'Zanubrutinib', 'Ziprasidone Hcl',
       'Zolmitriptan', 'Zolpidem Tartrate', 'Zonisamide'],
      dtype='object', length=875)

In [55]:
df.head()

,Prscrbr_NPI,Prscrbr_Type,Gnrc_Name,Tot_Clms
0,1003000126,Internal Medicine,Amlodipine Besylate,19.0
1,1003000126,Internal Medicine,Atorvastatin Calcium,11.0
2,1003000126,Internal Medicine,Apixaban,15.0
3,1003000126,Internal Medicine,Escitalopram Oxalate,16.0
4,1003000126,Internal Medicine,Hydrochlorothiazide,12.0


In [56]:
df['Gnrc_Name'].cat.codes

0            39
1            64
2            52
3           310
4           401
           ... 
24246830    860
24246831    364
24246832    720
24246833    869
24246834    873
Length: 24246835, dtype: int16

In [57]:
df['Gnrc_Name'].cat.categories[19]

'Albuterol Sulfate'

In [58]:
row_index = df['Prscrbr_NPI'].cat.codes
num_rows = df['Prscrbr_NPI'].cat.categories.size

col_index = df['Gnrc_Name'].cat.codes
num_cols = df['Gnrc_Name'].cat.categories.size

values = df['Tot_Clms'].values

In [59]:
from scipy.sparse import csr_matrix

sparse_matrix = csr_matrix(
    (values, (row_index, col_index)),
    shape=(num_rows, num_cols),
)

In [60]:
sparse_matrix.shape

(537484, 875)

In [61]:
from scipy.sparse.linalg import svds

In [62]:
U, sigma, Vt = svds(sparse_matrix, k=20)

In [63]:
U.shape, sigma.shape, Vt.shape

((537484, 20), (20,), (20, 875))

In [64]:
np.savetxt(DATA_DIR / 'features.tsv', Vt.T, delimiter='\t')

In [65]:
np.savetxt(
    DATA_DIR / 'labels.tsv',
    df['Gnrc_Name'].cat.categories.values,
    delimiter='\t',
    fmt='%s',
    encoding='utf-8',
)